## Cleaning and Analysis of ebay Car Sales data

The aim of the project is to clean and analyse the dataset of used cars from eBay *Kleinanzeigen*, a classifieds section of the German eBay website.

In [1]:
import numpy as np
import pandas as pd

 ## Step 1: Read the Data

Read the auto.csv file into pandas using different encoding techniques to find the one without throwing an error. Assign it to a variable autos

In [2]:
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [3]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


On the first look, below observations can be made:
- The dataset contains 20 columns most of which are strings and 371258 entries
- There are a few columns with null values but none have more than ~20% null values
- The column names uses camelCase than the preferred snake_case used in python, which means we can't just replace spaces with underscores.

## Step 2: Cleaning Data

We will start the data cleaning process one step at a time

### Cleaning Columns

We will be converting the column names from camelCase to snake_case and changing some of the column names to make it more descriptive

In [4]:
autos.columns
autos.rename(columns = {"dateCrawled":"date_crawled",
                        "offerType":"offer_type",
                        "vehicleType":"vehicle_type",
                        "yearOfRegistration":"registration_year",
                       "powerPS":"power_ps",
                       "monthOfRegistration":"registration_month",
                       "fuelType":"fuel_type",
                       "notRepairedDamage":"unrepaired_damage",
                       "dateCreated":"ad_created",
                       "nrOfPictures":"num_pictures",
                       "postalCode":"postal_code",
                       "lastSeen":"last_seen"}, inplace=True)

autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Data Exploration

We start exploring the data by using describe() so that we can find areas where we can being cleaning the data

In [5]:
autos.describe(include = "all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-04 16:40:33,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The following observations can be made:
 - *seller* and *offer_type* are very similar just differ by one value
 - *num_pictures* are 0 for all entries

The above columns are good candidates that can be dropped
 - *price* and *odometer* are stored as strings

These need to be changed to numericals
 - *registration_year* min value is 1000 and max value is 9000 which doesn't make sense
 - *regostration_month* has minimum value as 0 which is invlaid
    

### Dropping Columns with just one value

In [6]:
autos = autos.drop(["seller", "offer_type", "num_pictures"], axis = 1)

In [7]:
autos.shape

(50000, 17)

## Cleaning and converting numerical data stored as strings

To achieve this, we need to first clean the data of strings and convert to numerical data type

In [8]:
autos["price"].unique()

array(['$5,000', '$8,500', '$8,990', ..., '$385', '$22,200', '$16,995'],
      dtype=object)

In [9]:
autos["price"] = (autos["price"]
                  .str.replace("$","")
                  .str.replace(",","")
                  .astype(int))

In [10]:
autos["odometer"].unique()

array(['150,000km', '70,000km', '50,000km', '80,000km', '10,000km',
       '30,000km', '125,000km', '90,000km', '20,000km', '60,000km',
       '5,000km', '100,000km', '40,000km'], dtype=object)

In [11]:
autos["odometer"] = (autos["odometer"]
                  .str.replace("km","")
                  .str.replace(",","")
                  .astype(int))

In [12]:
autos.rename(columns = {"odometer" : "odometer_km"}, inplace = True)
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


### Exploring price and odometer_km

We explore the *price* and *odometer_km* columns to see if we can clean it further

In [13]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [14]:
autos["price"].value_counts().sort_index().head(50)

0      1421
1       156
2         3
3         1
5         2
8         1
9         1
10        7
11        2
12        3
13        2
14        1
15        2
17        3
18        1
20        4
25        5
29        1
30        7
35        1
40        6
45        4
47        1
49        4
50       49
55        2
59        1
60        9
65        5
66        1
70       10
75        5
79        1
80       15
89        1
90        5
99       19
100     134
110       3
111       2
115       2
117       1
120      39
122       1
125       8
129       1
130      15
135       1
139       1
140       9
Name: price, dtype: int64

In [15]:
autos["price"].value_counts().sort_index(ascending = False).head(50)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
169000      1
163991      1
163500      1
155000      1
151990      1
145000      1
139997      1
137999      1
135000      1
130000      1
129000      1
128000      1
120000      2
119900      1
119500      1
116000      1
115991      1
115000      1
114400      1
109999      1
105000      2
104900      1
99900       2
99000       2
98500       1
Name: price, dtype: int64

In [16]:
autos["price"].value_counts().head(20)

0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

We observe that most values are rounded, which might indicate that the sellers tend to round-up prices. Additionally, 1421 entries are 0 - given that this is only 2\% of the total cars, we might consider removing these. There are a number of listings with prices less than 50, and a few listings with prices more than 1 million. Given that eBay is an auction site, there could be items where the bid opens at 1. We will however remove items listed above 350000 since the prices seem to increase steadily after that.

In [17]:
autos = autos[autos["price"].between(1,350000)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

In [18]:
autos["odometer_km"].describe()

count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [19]:
autos["odometer_km"].value_counts().sort_index()

5000        836
10000       253
20000       762
30000       780
40000       815
50000      1012
60000      1155
70000      1217
80000      1415
90000      1734
100000     2115
125000     5057
150000    31414
Name: odometer_km, dtype: int64

From the above data we can see that all the values are rouded and the seller might have used Pre-set data. It also shows that there are more high mileage cars.

### Exploring the incorrect data columns

In [20]:
autos["registration_year"].unique()

array([2004, 1997, 2009, 2007, 2003, 2006, 1995, 1998, 2000, 2017, 2010,
       1999, 1982, 1990, 2015, 2014, 1996, 1992, 2002, 2012, 2011, 2005,
       2008, 1985, 2016, 1994, 1986, 2001, 2018, 2013, 1972, 1993, 1988,
       1989, 1973, 1967, 1976, 4500, 1987, 1991, 1983, 1960, 1969, 1950,
       1978, 1980, 1984, 1963, 1977, 1961, 1968, 1934, 1965, 1971, 1966,
       1979, 1981, 1970, 1974, 1910, 1975, 5000, 4100, 2019, 1956, 9999,
       6200, 1964, 1959, 1958, 1800, 1948, 1931, 1943, 1941, 1962, 1927,
       1937, 1929, 1000, 1957, 1952, 1111, 1955, 1939, 8888, 1954, 1938,
       2800, 5911, 1953, 1951, 4800, 1001, 9000], dtype=int64)

We can see that there are many values in *registration_year* column which are invalid.The listings are from 2016 (as we will see later), any vehicle with a registartion year above 2016 is inaccurate.

In [21]:
autos.loc[autos["registration_year"] < 1910, "registration_year"]

10556    1800
22316    1000
24511    1111
32585    1800
49283    1001
Name: registration_year, dtype: int64

These values can be removed. So let's only consider values which have registration year between 1910 and 2016

In [22]:
autos = autos[autos["registration_year"].between(1910,2016)]
autos["registration_year"].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

We can see that most vehicles were registered in the 2000's. Now let's look at *registration_month*

In [23]:
autos["registration_month"].describe()

count    46681.000000
mean         5.827125
std          3.670300
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

In [24]:
autos["registration_month"].value_counts()

3     4838
6     4119
0     4026
4     3894
5     3877
7     3722
10    3495
12    3262
9     3243
11    3229
1     3105
8     3021
2     2850
Name: registration_month, dtype: int64

There are over 4000 entires with 0 value for months which is invalid. It would be suitable to remove these entries.

In [25]:
autos = autos[~(autos["registration_month"] == 0)]
autos["registration_month"].value_counts()

3     4838
6     4119
4     3894
5     3877
7     3722
10    3495
12    3262
9     3243
11    3229
1     3105
8     3021
2     2850
Name: registration_month, dtype: int64

### Exploring the Date columns

In the dataset, the below columns represent data values

- date_crawled
- registration_year
- registration_month
- ad_created
- last_seen

Some of these columns were created by the crawler and some were scrapted from the website. *date_crawled*, *last_seen* and *ad_created* are all identifed by strings as pandas

In [26]:
autos[["date_crawled", "last_seen", "ad_created"]].head()

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


The dates are stored in the format YYYY-MM-DD, the first 10 characters represent the date

In [27]:
autos["date_crawled"].str[:10].value_counts(normalize = True, dropna = False).sort_values(ascending = False)

2016-04-03    0.039222
2016-03-20    0.038167
2016-03-21    0.037299
2016-03-12    0.036971
2016-03-14    0.036619
2016-04-04    0.036596
2016-03-07    0.036362
2016-04-02    0.035916
2016-03-28    0.034697
2016-03-19    0.034416
2016-04-01    0.034345
2016-03-15    0.033900
2016-03-29    0.033876
2016-03-30    0.033431
2016-03-08    0.033220
2016-03-09    0.033126
2016-03-11    0.032986
2016-03-22    0.032728
2016-03-23    0.032306
2016-03-26    0.032259
2016-03-10    0.032212
2016-03-31    0.031790
2016-03-17    0.031368
2016-03-25    0.031087
2016-03-27    0.030946
2016-03-16    0.029399
2016-03-24    0.029211
2016-03-05    0.025062
2016-03-13    0.015637
2016-03-06    0.014301
2016-04-05    0.013082
2016-03-18    0.012894
2016-04-06    0.003118
2016-04-07    0.001454
Name: date_crawled, dtype: float64

All the entries are from March and April 2016. The site was probably crawled around that time. The number of listings are roughly uniform.

In [28]:
autos["ad_created"].str[:10].value_counts(normalize = True, dropna = False).sort_index()

2015-06-11    0.000023
2015-08-10    0.000023
2015-09-09    0.000023
2015-11-10    0.000023
2015-12-05    0.000023
                ...   
2016-04-03    0.039386
2016-04-04    0.036924
2016-04-05    0.011886
2016-04-06    0.003212
2016-04-07    0.001289
Name: ad_created, Length: 74, dtype: float64

Ranges of date spread over 10 months but most of the them are in April and March 2016.

In [29]:
autos["last_seen"].str[:10].value_counts(normalize = True, dropna = False).sort_index()

2016-03-05    0.001102
2016-03-06    0.003939
2016-03-07    0.005392
2016-03-08    0.007268
2016-03-09    0.009659
2016-03-10    0.010597
2016-03-11    0.012331
2016-03-12    0.023655
2016-03-13    0.008557
2016-03-14    0.012519
2016-03-15    0.015872
2016-03-16    0.016294
2016-03-17    0.027617
2016-03-18    0.007385
2016-03-19    0.015450
2016-03-20    0.020302
2016-03-21    0.020537
2016-03-22    0.020537
2016-03-23    0.018239
2016-03-24    0.019505
2016-03-25    0.018614
2016-03-26    0.016645
2016-03-27    0.015168
2016-03-28    0.020443
2016-03-29    0.021944
2016-03-30    0.024288
2016-03-31    0.023514
2016-04-01    0.023374
2016-04-02    0.025062
2016-04-03    0.025038
2016-04-04    0.023678
2016-04-05    0.127300
2016-04-06    0.224358
2016-04-07    0.133818
Name: last_seen, dtype: float64

The last seen values record the date the last time the listing was seen. This will give us an idea on when the card was sold.
Note that last three days show a disproportionate increase of 6-10x. This most likely has to do with the crawling period ending and not with the car sales.

### Exploring Price by Brand

In [30]:
autos["brand"].value_counts(normalize = True)

volkswagen        0.208229
bmw               0.112015
opel              0.103903
mercedes_benz     0.099730
audi              0.088032
ford              0.068738
renault           0.046466
peugeot           0.030008
fiat              0.025437
seat              0.018357
skoda             0.017044
nissan            0.015473
mazda             0.014934
smart             0.014441
citroen           0.013808
toyota            0.013293
hyundai           0.010245
mini              0.009354
sonstige_autos    0.009331
volvo             0.009284
mitsubishi        0.008018
honda             0.007830
kia               0.007479
alfa_romeo        0.006611
porsche           0.006400
suzuki            0.006002
chevrolet         0.005744
chrysler          0.003470
dacia             0.002766
jeep              0.002368
daihatsu          0.002344
land_rover        0.002180
subaru            0.002016
saab              0.001711
jaguar            0.001571
daewoo            0.001500
rover             0.001313
t

Volkswagen is the most popular brand with approximately double the cars sales compared to the next brand.

There are lot of brands that don't have a significant percentage of listings , so we will limit our analysis to brands representing more than 3% of total listings.

In [31]:
brand_counts = autos["brand"].value_counts(normalize = True)
common_brands = brand_counts[brand_counts > 0.03].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot'],
      dtype='object')


In [32]:
brand_mean_prices = {}

for brand in common_brands:
    brands_only = autos[autos["brand"] == brand]
    mean_price = brands_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
brand_mean_prices

{'volkswagen': 5752,
 'bmw': 8652,
 'opel': 3167,
 'mercedes_benz': 8882,
 'audi': 9776,
 'ford': 3957,
 'renault': 2616,
 'peugeot': 3221}

We can see a gradual price gap above:
- Audi, BMW and Benz are expensive
- Volkswagen is in the mid range (Reason for it's popularity)
- Opel, Renault and Peugeotare in the lower range of prices

We can also notice that the expensive cars comes second in the popularity list. The ranking order would be 1) Mid range cars 2) Expensive cars 3) Lower range cars

### Exploring Mileage

In [35]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series,columns = [mean_price])

,3221.411719
volkswagen,5752
bmw,8652
opel,3167
mercedes_benz,8882
audi,9776
ford,3957
renault,2616
peugeot,3221


In [38]:
brand_mean_mileage = {}

for brand in common_brands:
    brands_only = autos[autos["brand"] == brand]
    mean_mileage = brands_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
brand_mean_mileage    

{'volkswagen': 128183,
 'bmw': 132489,
 'opel': 128752,
 'mercedes_benz': 130927,
 'audi': 128593,
 'ford': 124009,
 'renault': 127820,
 'peugeot': 126269}

In [48]:
mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending = False)
mean_price = pd.Series(brand_mean_prices).sort_values(ascending = False)
popularity = pd.Series(autos["brand"].value_counts(normalize=True).sort_values(ascending = False))

brand_info = pd.DataFrame(mean_mileage,columns = ["mean_mileage"])
brand_info["mean_price"] = mean_price
brand_info["popularity"] = popularity
brand_info

,mean_mileage,mean_price,popularity
bmw,132489,8652,0.112015
mercedes_benz,130927,8882,0.099730
opel,128752,3167,0.103903
audi,128593,9776,0.088032
volkswagen,128183,5752,0.208229
renault,127820,2616,0.046466
peugeot,126269,3221,0.030008
ford,124009,3957,0.068738


From our analysis, We can see that the expensive cars have more mileage and the popularity is roughly proprtional to it